In [1]:
import glob
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from math import ceil
import numpy as np
import os
import pandas as pd
from random import randint, sample
from scipy.stats import chi2, chi2_contingency, gaussian_kde
from sklearn.neighbors import KernelDensity
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric.kernel_density import KDEMultivariate
import torch

plt.rcParams.update({'figure.max_open_warning': 0})
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams["font.family"] = "Times New Roman"
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.rc('legend', fontsize=15)


In [2]:
path = "./width_exp_T10/"
path = "./width_exp_T100/"

dataset = 'mnist'
loss = 'NLL'

nets = {}

def load_net(filename):
    net = torch.load(path + filename + '/net.pyT',map_location='cpu')
    return net

def load_all():
    results = {}
    hist_tr = {}
    hist_te = {}
    nets    = {}

    widths = [100, 200, 500, 1000, 2000, 5000, 10000, 50000, 100000]
    betas = [0.0, 0.25, 0.5, 0.75, 1.0]
    batch_sizes = [10, 100, 1000, 5000, 10000]
    gammas = [1.0, 0.1, 0.01, 0.001]
    grid = itertools.product(widths, betas, batch_sizes, gammas)

    tot = 0
    for w, b, bs, g in grid:
        b = str(b)
        g = str(g)
        filename = '{:04d}_00_mnist_NLL_{:s}_1.0_{:d}_{:s}'.format(w, b, bs, g)
        f = path + filename + '/training_history.hist'
        if os.path.isfile(f):
            tot += 1
            print(filename)
            #results[folder] = torch.load(f)
            #hist_tr[folder] = torch.load(base + '/evaluation_history_TRAIN.hist',map_location='cpu')
            #hist_te[folder] = torch.load(base + '/evaluation_history_TEST.hist',map_location='cpu')
            nets[filename]    = torch.load(path + filename + '/net.pyT',map_location='cpu')
    print('{:d} networkds loaded'.format(tot))
    
    return nets


In [3]:
def kde_scipy(x, x_grid, bandwidth=0.2, **kwargs):
    """Kernel Density Estimation with Scipy"""
    # Note that scipy weights its bandwidth by the covariance of the
    # input data.  To make the results comparable to the other methods,
    # we divide the bandwidth by the sample standard deviation here.
    kde = gaussian_kde(x, bw_method=bandwidth / x.std(ddof=1), **kwargs)
    return kde.evaluate(x_grid)


def kde_statsmodels_u(x, x_grid, bandwidth=0.2, **kwargs):
    """Univariate Kernel Density Estimation with Statsmodels"""
    kde = KDEUnivariate(x)
    kde.fit(bw=bandwidth, **kwargs)
    return kde.evaluate(x_grid)
    
    
def kde_statsmodels_m(x, x_grid, bandwidth=0.2, **kwargs):
    """Multivariate Kernel Density Estimation with Statsmodels"""
    kde = KDEMultivariate(x, bw=bandwidth * np.ones_like(x),
                          var_type='c', **kwargs)
    return kde.pdf(x_grid)


def kde_sklearn(x, x_grid, bandwidth=0.2, **kwargs):
    """Kernel Density Estimation with Scikit-learn"""
    kde_skl = KernelDensity(bandwidth=bandwidth, **kwargs)
    kde_skl.fit(x[:, np.newaxis])
    # score_samples() returns the log-likelihood of the samples
    log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
    return np.exp(log_pdf)

def kde_sklearn_2(x, x_grid, **kwargs):
    bandwidths = 10 ** np.linspace(-1, 1, 100)
    grid = GridSearchCV(KernelDensity(kernel='gaussian'),
                    {'bandwidth': bandwidths},
                    cv=LeaveOneOut())
    grid.fit(x[:, None]);
    print(grid)
    kde_skl = KernelDensity(bandwidth=grid.bandwidth, **kwargs)
    kde_skl.fit(x[:, np.newaxis])
    # score_samples() returns the log-likelihood of the samples
    log_pdf = kde_skl.score_samples(x_grid[:, np.newaxis])
    return np.exp(log_pdf)

In [4]:
def weights_net(net, coords=False, coord=0):
    weights = []
    l = 0
    for p in net.parameters():
        if l == 0:
            weight = p.detach().numpy()
            if not coords:
                weight = weight[:,coord]
        else:
            weight = p.detach().numpy()
            if not coords:
                weight = weight[0,:]
        #weight = weight[np.abs(weight)<2]
        #weight = weight.flatten()
        weights.append(weight)
        l += 1
    return weights

In [5]:
def limit_density(beta, alpha=0.0, bound=0.25, one=False):
    beta = '0.25'
    batch_size = 1000
    width = 50000
    bw = 0.005 #0.002
    bw2 = 0.02
    if alpha > 0:
        beta = '0.5'
        width = 50000
    if one:
        beta = '1.0'
        batch_size = one
        bw2 = 0.02
    filename = '{:04d}_00_mnist_NLL_{:s}_{:s}_{:d}_1.0'.format(width, beta, str(1.0 - alpha), batch_size)
    print('limit', filename)
    net = load_net(filename)
    weights = weights_net(net)
    xgrid = np.linspace(-bound, bound, 1000)
    pdf1 = kde_sklearn(weights[0], xgrid, bandwidth=bw)
    pdf2 = kde_sklearn(weights[1], xgrid, bandwidth=bw2)
    
    return xgrid, pdf1, pdf2

def density(weights, bound):
    xgrid = np.linspace(-bound, bound, 1000)
    pdf = kde_sklearn(weights, xgrid, bandwidth=0.002)
    return xgrid, pdf


def empirical_measure_fig(width, beta, batch_size, alpha=0.0, layer=1, gamma=1.0, xbound=None, ybound=None, limit=True, one=False, suffix=''):
    beta = str(beta)
    gamma0 = str(gamma)
    filename = '{:04d}_00_mnist_NLL_{:s}_{:s}_{:d}_{:s}{:s}'.format(width, beta, str(1.0 - alpha), batch_size, gamma0, suffix)
    net = load_net(filename)
    print(filename)

    weights = weights_net(net, coords=False)
    w = weights[layer - 1]
    print(w.shape)
    print(np.max(w), np.min(w), np.mean(w), np.std(w))
    if limit:
        xgrid, pdf1, pdf2 = limit_density(beta, alpha, xbound, one)
        pdf = pdf1 if (layer == 1) else pdf2
        plt.plot(xgrid, pdf, color='red', lw=3, label=r'limit density') #  for $\beta$<1
        #### limit for alpha=0
        if alpha > 0 and beta == '1.0':
            xgrid, pdf1, pdf2 = limit_density(beta, 0.0, xbound, one)
            pdf = pdf1 if (layer == 1) else pdf2
            plt.plot(xgrid, pdf, 'g--', lw=3, label=r'limit density for $\alpha$=0')        
        
    if beta == '1.0':
        plt.hist(w, 300, density=True, label='histogram', range=(-xbound, xbound))
    elif batch_size == 1:
        plt.hist(w, 300, density=True, label='histogram', range=(-xbound, xbound))
    else:
        plt.hist(w, 300, density=True, label='histogram')
    if xbound is not None:
        plt.xlim(-xbound, xbound)
    if ybound is not None:
        plt.ylim(0, ybound)
    # plt.title(r'Layer {:d}, $\beta$ = {:s}, N = {:d}, M = {:d}'.format(layer, beta, width, batch_size))
    plt.title(r'$M={:d}$'.format(batch_size), fontsize=18)
    #plt.title(r'$N={:d}$'.format(width), fontsize=18)
    #plt.title(r'$\gamma={:s}$'.format(str(gamma)), fontsize=18)
    plt.legend(loc='upper right')
    

def empirical_measure_both_fig(width, beta, batch_size, alpha=0.0, gamma=1.0, xbounds=[0.1,1.], ybound=None, limit=True, save=False):
    plt.figure(figsize=(12,3))
    
    plt.subplot(121)
    empirical_measure_fig(width, beta, batch_size, alpha, 1, gamma, xbounds[0], ybound, limit)
    plt.subplot(122)
    empirical_measure_fig(width, beta, batch_size, alpha, 2, gamma, xbounds[1], ybound, limit)
      
    if save:
        plt.savefig('Figures/empirical_mean__beta{:s}_width{:d}_batch{:d}.png'.format(beta, width, batch_size))
    
def convergence_empirical_measure(beta, batch_size, alpha=0.0, xbounds=[0.1,1.], ybound=None, limit=True, save=False):
    widths = [500, 1000, 5000, 10000, 50000, 100000]
    for w in widths:
         empirical_measure_both_fig(w, beta, batch_size, alpha, 1.0, xbounds, ybound, limit, save)
            
def convergence_empirical_measure_gammas(width, batch_size=100, alpha=0.0, xbounds=[0.1,1.], ybound=None, limit=True, save=False):
    gammas = [1.0, 0.1, 0.01, 0.001]
    beta = 1.0
    for g in gammas:
        plt.figure(figsize=(12,3))
        plt.subplot(121)
        empirical_measure_fig(width, beta, batch_size, alpha, 1, g, xbounds[0], ybound, limit)
        plt.subplot(122)
        empirical_measure_fig(width, beta, batch_size, alpha, 2, g, xbounds[1], ybound, limit)


In [6]:
import warnings
warnings.filterwarnings("ignore")

# CV of empirical measures for paper
def cv_empirical(beta, batch_size, alpha=0.0, layer=1, xbound=0.25, ybound=None, limit=True, one=False, save=False, large=False, small=False):
    print('beta={:s} batch_size={:d} alpha={:s} layer={:d}'.format(str(beta), batch_size, str(alpha), layer))
    #widths = [500, 1000, 5000, 10000, 50000]
    widths = [500, 5000, 50000]
    if large:
        widths = [1000, 10000, 50000, 100000, 1000000]
        widths = [500, 1000, 5000, 10000, 50000, 1000000]
    if small:
        widths = [500, 1000, 5000, 10000]
    plt.figure(figsize=(12,3))
    for i, w in enumerate(widths):
        plt.subplot(1, len(widths), 1 + i)
        empirical_measure_fig(w, beta, batch_size, alpha, layer, 1.0, xbound, ybound, limit, one)
    
    if save:
        plt.savefig('Figures_ok/CV_emp_measure_beta{:s}_alpha{:s}_batchsize{:d}_layer{:d}.jpg'.format(str(beta).replace('.',''), str(alpha).replace('.',''), batch_size, layer), dpi=500)
        
        
def cv_empirical_gammas(batch_size=100, alpha=0.0, layer=1, xbound=0.25, ybound=None, limit=True, save=False):
    gammas = [1.0, 0.1, 0.01]
    w = 10000
    plt.figure(figsize=(12,3))
    for i, g in enumerate(gammas):
        plt.subplot(1, len(gammas), 1 + i)
        empirical_measure_fig(w, 1.0, batch_size, alpha, layer, g, xbound, ybound, limit)
    
    if save:
        plt.savefig('Figures_ok/CV_emp_measure_gammas_alpha{:s}_batchsize{:d}_layer{:d}.jpg'.format(str(alpha).replace('.',''), batch_size, layer), dpi=500)
        
def cv_empirical_bs(beta, alpha=0.0, layer=1, gamma=1.0, xbound=0.25, ybound=None, limit=True, save=False, small=False, suffix=''):
    #batch_sizes = [1, 10, 100, 1000, 10000]
    batch_sizes = [1, 10, 100, 1000]
    if small:
        batch_sizes=[1, 10, 100]
    w = 10000
    plt.figure(figsize=(12,3))
    for i, bs in enumerate(batch_sizes):
        plt.subplot(1, len(batch_sizes), 1 + i)
        empirical_measure_fig(w, beta, bs, alpha, layer, gamma, xbound, ybound, limit, suffix=suffix)
    
    if save:
        plt.savefig('Figures_ok/CV_emp_measure_BS_beta{:s}_alpha{:s}_width{:d}_layer{:d}.jpg'.format(str(beta).replace('.',''), str(alpha).replace('.',''), w, layer), dpi=500)
    
save = False
# CV empirical distributions alpha = 0 BS = 100
# cv_empirical(0.25, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.25, 100, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.5, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 100, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.75, 100, layer=1, xbound=0.2, ybound=20, save=save, limit=True)
# cv_empirical(0.75, 100, layer=2, xbound=2.5, ybound=1.5, save=save, limit=True)
# cv_empirical(1.0, 100, layer=1, xbound=1., ybound=6, one=100, save=save, limit=True)
# cv_empirical(1.0, 100, layer=2, xbound=4, ybound=1., one=100, save=save, limit=True)

# CV empirical distributions alpha = 0 BS = 10
# cv_empirical(0.25, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.25, 10, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.5, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 10, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.75, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 10, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(1.0, 10, layer=1, xbound=2, ybound=None, one=10, save=save)
# cv_empirical(1.0, 10, layer=2, xbound=4, ybound=1., one=10, save=save)

# CV empirical distributions alpha = 0.25 BS = 100
#cv_empirical(0.25, 100, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
#cv_empirical(0.25, 100, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.5, 100, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 100, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.75, 100, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 100, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(1.0, 100, alpha=0.25, layer=1, xbound=1, ybound=10, one=100, save=save)
# cv_empirical(1.0, 100, alpha=0.25, layer=2, xbound=4, ybound=1.6, one=100, save=save)

# CV empirical distributions alpha = 0.25 BS = 10
#cv_empirical(0.25, 10, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
#cv_empirical(0.25, 10, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.5, 10, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 10, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(0.75, 10, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 10, alpha=0.25, layer=2, xbound=2.5, ybound=1.5, save=save)
# cv_empirical(1.0, 10, alpha=0.25, layer=1, xbound=2, ybound=None, one=10, save=save)
# cv_empirical(1.0, 10, alpha=0.25, layer=2, xbound=4, ybound=1., one=10, save=save)

# gamma CV
# cv_empirical_gammas(batch_size=10, layer=1, xbound=0.2, ybound=20, save=save, limit=True)
# cv_empirical_gammas(batch_size=10, layer=2, xbound=4, ybound=1.2, save=save)

# cv_empirical_bs(beta=1.0, alpha=0.0, layer=1, save=save, limit=False, small=True)
# cv_empirical_bs(beta=1.0, alpha=0.0, layer=2, ybound=1, save=save, limit=False, small=True)
# cv_empirical_bs(beta=1.0, alpha=0.25, layer=1,xbound=2, save=save)
# cv_empirical_bs(beta=1.0, alpha=0.25, layer=2, xbound=4, ybound=1, save=save)
# cv_empirical_bs(beta=0.75, alpha=0.0, layer=1,xbound=0.2, ybound=20, save=save, small=True)
# cv_empirical_bs(beta=0.75, alpha=0.0, layer=2, xbound=2.5, ybound=1, save=save, small=True)
# cv_empirical_bs(beta=0.75, alpha=0.25, layer=1,xbound=0.2, save=save)
# cv_empirical_bs(beta=0.75, alpha=0.25, layer=2, xbound=2.5, ybound=1, save=save)

# cv_empirical(0.75, 1000, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 1000, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 100, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 10, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 10, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 100, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 10, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 10, alpha=0.25, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.75, 100, alpha=0.25, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.75, 100, alpha=0.25, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.5, 1000, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 1000, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.5, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 100, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.5, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 10, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(0.5, 1, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(0.5, 1, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(1.0, 1000, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(1.0, 1000, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(1.0, 100, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(1.0, 100, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(1.0, 10, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(1.0, 10, layer=2, xbound=1.5, ybound=2.5, save=save)
# cv_empirical(1.0, 1, layer=1, xbound=0.2, ybound=20, save=save)
# cv_empirical(1.0, 1, layer=2, xbound=1.5, ybound=2.5, save=save)
#cv_empirical(1.0, 100, layer=1, xbound=1.0, ybound=6, one=100, save=save)
#cv_empirical(1.0, 100, layer=2, xbound=1.5, ybound=2.5, one=100, save=save)
#cv_empirical(1.0, 1, layer=1, xbound=1.0, ybound=6, one=1, save=save)
#cv_empirical(1.0, 1, layer=2, xbound=1.5, ybound=2.5, one=1, save=save)






In [7]:
# fixed_init
def empirical_measure_fig_fixed_init(width, beta, batch_size, nb, alpha=0.0, layer=1, gamma=1.0, xboundmin=None, xboundmax=None, ybound=None, limit=True, one=False):
    beta = str(beta)
    gamma0 = str(gamma)
    filename = '{:04d}_00_mnist_NLL_{:s}_{:s}_{:d}_{:s}_fixed_init_nb_{:d}'.format(width, beta, str(1.0 - alpha), batch_size, gamma0, nb)
    net = load_net(filename)
    print(filename)

    weights = weights_net(net, coords=False)
    w = weights[layer - 1]
    print(w.shape)
    print(np.max(w), np.min(w), np.mean(w), np.std(w))
    plt.plot([np.mean(w), np.mean(w)],[0,1], linewidth=2)
    if xboundmax is not None:
        plt.xlim(xboundmin, xboundmax)
    if ybound is not None:
        plt.ylim(0, ybound)
    return np.mean(w)
    
def all_fixed_init(beta, xboundmin=-0.2, xboundmax=0.2):
    width = 1000000
    batch_size = 100
    begin = 2
    end = 9
    list_vals = []
    for nb in range(begin, end + 1):
        if nb != 5:
            val = empirical_measure_fig_fixed_init(width, beta, batch_size, nb, xboundmin=xboundmin, xboundmax=xboundmax)
            list_vals.append(val)
    list_vals = np.array(list_vals)
    m = np.mean(list_vals)
    s = np.std(list_vals)
    print("beta= {:s}".format(str(beta)))
    print(m, s)

save = False
display = False
if display:
    plt.figure(figsize=(10,3))
    plt.subplot(1,2,1)
    all_fixed_init(0.75, xboundmin=-0.01, xboundmax=0.12)
    plt.title(r'$\beta=0.75$', fontsize=18)
    plt.subplot(1,2,2)
    all_fixed_init(1.0, xboundmin=-0.01, xboundmax=0.12)
    plt.title(r'$\beta=1.0$', fontsize=18)
    if save:
        plt.savefig('Figures_ok/regimes.jpg', dpi=2000)